In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from os import fdopen, remove, walk
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

In [3]:
from datetime import date

In [4]:
DATASET_PATH = "/Users/noopy/covid19_unknown_spread/dataset"
datasets = glob.glob(f"{DATASET_PATH}/*.csv")
datasets[:5]

['/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_31_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_30_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_17_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_21_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_7_04_.csv']

In [5]:
# get oldest file in the dataset folder
import os, heapq
def newst_files_in_tree(rootfolder, count=1, extension=".csv"):
    return heapq.nlargest(count,
        (os.path.join(dirname, filename)
        for dirname, dirnames, filenames in os.walk(rootfolder)
        for filename in filenames
        if filename.endswith(extension)),
        key=lambda fn: os.stat(fn).st_mtime)

In [6]:
newst_csv = newst_files_in_tree(DATASET_PATH)[0]

In [7]:
df_temp = pd.read_csv(newst_csv, encoding="utf-8")
df_temp.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
3656,595,10441,4.09.,성북구,-,확인 중,퇴원
266,3985,20258,9.01.,강남구,-,강남구 소재 아파트,NaN
768,3483,18857,8.27.,강동구,-,확인 중,NaN
1748,2503,16411,8.20.,은평구,-,확인 중,퇴원
61,4190,20823,9.03.,강동구,-,기타 확진자 접촉,NaN
2037,2214,15969,8.18.,기타,-,성북구 사랑제일교회 관련,NaN
2590,1661,14550,8.7.,서초구,-,서초구 확진자 접촉,퇴원
145,4106,20555,9.02.,서초구,-,중구 소재 은행,NaN
3706,545,10233,4.04.,송파구,-,#9765 접촉,퇴원
2350,1901,15132,8.15.,노원구,-,성북구 사랑제일교회 관련,퇴원


In [8]:
# overwrite whatever cell value that contains "확인" as "확인 중"
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"] = "확인 중"

# check whether those two arrays are combined
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"].unique()

array(['확인 중'], dtype=object)

In [9]:
df_temp["접촉력"] = df_temp["접촉력"].str.replace("관련", "")
df_temp["접촉력"] = df_temp["접촉력"].str.strip()

In [10]:
infection_paths = df_temp["접촉력"].unique()
print(len(infection_paths))
infection_paths

223


array(['송파구 소재 병원', '노원구 빛가온교회', '강동구 소재 병원', '성북구 사랑제일교회', '8.15도심집회',
       '확인 중', '도봉구 운동시설', '타시도 확진자 접촉', '기타 확진자 접촉', '다래경매', '노원구 기도모임',
       '동작구 카드 발급업체', '광진구 소재병원', '성북구 체대입시', '성북구 요양시설', '중구 소재 은행',
       '중랑구 소재 체육시설', '관악구 판매업소', '동작구 소재 서울신학교', '용인시 우리제일교회',
       '노원구 손해보험', '구로구 보성운수', '강서구 서울대효요양병원', '서초구 장애인교육시설', '서대문구 지인모임',
       '강북구 일가족', '중구 하나은행본점', '강서구 보안회사', '롯데리아 종사자 모임', '해외 접촉 추정',
       '8.15도심집회(순복음 강북교회)', '영등포구 권능교회', '구로구 아파트', '강남구 소재 아파트',
       '제주 게스트하우스', '성북구 벧엘장로교회', '중앙보훈병원', '관악구 김혜근의원', '동대문구 sk탁구클럽',
       '동작구 스터디카페', '극단 산', 'KT가좌지사', '군인권센터', '동작구 요양시설', '여의도 순복음교회',
       '중구 보험회사(현대해상)', '강서구 병원', '종로구 혜화경찰서', '양천구 되새김교회', '현대커머셜',
       '8.15도심집회(녹색병원)', '영등포 IFC몰 오케스트로', '고양시 반석교회(케네디상가)', '롯데 자산개발',
       '골드트레인', '한양대병원', '은평구 헤어콕', '순복음 강북교회', '서대문구 지인 모임', '강동구 어린이집',
       '광화문집회', '은평구 성경공부모임', '관악구 요양병원', '고대 안암병원', '마포구 푸본생명콜센터',
       '성동구 가족', '양천구 되새김 교회', '중구 통일상가', '고양시 반석교회  (케네디상가)',
       '강남구 판매업소(

In [11]:
df_date = df_temp.sort_values(["연번"], ascending=False)
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9.04.,타시,-,송파구 소재 병원,NaN
1,4250,20979,9.04.,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9.04.,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9.04.,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9.03.,성북구,-,8.15도심집회,NaN


In [12]:
df_date["확진일"] = df_date["확진일"].str.replace(".", "-")
df_date["확진일"] = df_date["확진일"].str[:-1]
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,9-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9-03,성북구,-,8.15도심집회,NaN


In [13]:
df_date["확진일"] = "2020-0" + df_date["확진일"]
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [14]:
df_date.sample(5)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
3733,518,10134,2020-04-3,송파구,미국,해외 접촉 추정,퇴원
2323,1928,15157,2020-08-15,성북구,-,성북구 사랑제일교회,NaN
3232,1019,11875,2020-06-09,관악구,-,리치웨이,퇴원
785,3466,18901,2020-08-27,은평구,-,동작구 카드 발급업체,사망
2810,1441,13547,2020-07-14,동작구,-,영등포구 확진자 접촉,퇴원


In [15]:
df = df_date.copy()
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [16]:
df.loc[(df["확진일"].str.len() < 10), '확진일'] = df.loc[(df["확진일"].str.len() < 10), '확진일'].str.replace("-0", "-")
df.loc[(df["확진일"].str.len() < 10), '확진일'] = df.loc[(df["확진일"].str.len() < 10), '확진일'].str.replace("-", "-0")
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [17]:
df.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
1183,3068,17592,2020-08-24,노원구,-,성북구 사랑제일교회,NaN
1567,2684,16917,2020-08-21,송파구,-,광화문집회,NaN
3041,1210,12397,2020-06-20,성동구,-,금천구 도정기 회사,퇴원
1435,2816,17148,2020-08-22,영등포구,-,여의도 순복음교회,퇴원
1704,2547,16488,2020-08-20,강서구,-,성북구 체대입시,NaN
988,3263,18651,2020-08-26,강서구,-,확인 중,NaN
2799,1452,13646,2020-07-16,도봉구,-,강남구 사무실,퇴원
1020,3231,18216,2020-08-25,종로구,-,기타 확진자 접촉,NaN
1940,2311,15971,2020-08-18,용산구,-,확인 중,퇴원
410,3841,19918,2020-08-30,관악구,-,동작구 소재 서울신학교,NaN


In [18]:
df.to_csv("./dataset_predict/df_wrangle.csv", index=False)

In [19]:
temp = df["확진일"].value_counts().rename_axis('date').reset_index(name='new_confirmed')
temp.head()

,date,new_confirmed
0,2020-08-26,153
1,2020-08-18,151
2,2020-08-27,146
3,2020-08-15,146
4,2020-08-22,140


In [20]:
df_predict = temp.sort_values(by="date")
df_predict.head()

,date,new_confirmed
189,2020-01-24,1
170,2020-01-30,3
163,2020-01-31,3
192,2020-02-02,1
174,2020-02-05,2


In [21]:
df_predict["date"] = pd.to_datetime(df_predict["date"])

In [22]:
df_predict.index = pd.DatetimeIndex(df_predict["date"])
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1
2020-01-30,2020-01-30,3
2020-01-31,2020-01-31,3
2020-02-02,2020-02-02,1
2020-02-05,2020-02-05,2


In [23]:
df_predict = df_predict.resample('D').max()
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,NaT,NaN
2020-01-26,NaT,NaN
2020-01-27,NaT,NaN
2020-01-28,NaT,NaN


In [24]:
df_predict["date"] = df_predict.index
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,2020-01-25,NaN
2020-01-26,2020-01-26,NaN
2020-01-27,2020-01-27,NaN
2020-01-28,2020-01-28,NaN


In [25]:
df_predict = df_predict.fillna(0)
df_predict

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,2020-01-25,0.0
2020-01-26,2020-01-26,0.0
2020-01-27,2020-01-27,0.0
2020-01-28,2020-01-28,0.0
...,...,...
2020-08-31,2020-08-31,94.0
2020-09-01,2020-09-01,101.0
2020-09-02,2020-09-02,81.0


In [26]:
df_predict.to_csv("dataset_predict/df_predict.csv", index=False)

In [27]:
df_predict =  pd.read_csv("dataset_predict/df_predict.csv", encoding="utf-8")

In [28]:
df_untracked = df[df["접촉력"]=="확인 중"]
df_untracked

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
5,4246,미부여,2020-09-03,관악구,-,확인 중,NaN
6,4245,미부여,2020-09-03,마포구,-,확인 중,NaN
7,4244,미부여,2020-08-30,기타,-,확인 중,NaN
8,4243,미부여,2020-09-02,마포구,-,확인 중,NaN
9,4242,미부여,2020-09-03,성동구,-,확인 중,NaN
...,...,...,...,...,...,...,...
4195,56,1768,2020-02-25,고양시,-,확인 중,퇴원
4209,42,1370,2020-02-25,노원구,-,확인 중,퇴원
4212,39,924,2020-02-25,평택,-,확인 중,퇴원
4213,38,907,2020-02-25,관악구,-,확인 중,퇴원


In [29]:
df_untracked_temp = df_untracked.groupby(by=["확진일"]).size()
df_untracked_temp = df_untracked_temp.rename_axis('date').reset_index(name='new_untracked')
df_untracked_temp

,date,new_untracked
0,2020-02-25,6
1,2020-02-26,1
2,2020-02-27,2
3,2020-02-28,2
4,2020-03-02,2
...,...,...
119,2020-08-31,16
120,2020-09-01,18
121,2020-09-02,17
122,2020-09-03,11


In [30]:
df_untracked_no = df_untracked_temp

In [31]:
df_merged = pd.merge(df_predict, df_untracked_no, how="left", on="date")
df_merged = df_merged.fillna(0)
df_merged.head(15)

,date,new_confirmed,new_untracked
0,2020-01-24,1.0,0.0
1,2020-01-25,0.0,0.0
2,2020-01-26,0.0,0.0
3,2020-01-27,0.0,0.0
4,2020-01-28,0.0,0.0
5,2020-01-29,0.0,0.0
6,2020-01-30,3.0,0.0
7,2020-01-31,3.0,0.0
8,2020-02-01,0.0,0.0
9,2020-02-02,1.0,0.0


In [32]:
df_merged.tail(15)

,date,new_confirmed,new_untracked
210,2020-08-21,128.0,32.0
211,2020-08-22,140.0,44.0
212,2020-08-23,97.0,25.0
213,2020-08-24,134.0,45.0
214,2020-08-25,113.0,34.0
215,2020-08-26,153.0,45.0
216,2020-08-27,146.0,34.0
217,2020-08-28,127.0,35.0
218,2020-08-29,114.0,20.0
219,2020-08-30,95.0,20.0


In [33]:
# list_infection_paths_no = []
for index, row in df_merged.iterrows():
    filter_end_date = row["date"]
    # print(filter_end_date)
    df_filtered = df.loc[df['확진일'] <= filter_end_date]
    
    # 확진자 접촉력이 밝혀진 경우
    infection_paths_known = df_filtered["접촉력"].unique()
    
    # "기타 확진자 접촉" 항목 485건 / R0
    contact_no = len(df_filtered[df_filtered["접촉력"].str.contains("기타 확진자 접촉")])/1.5
    
    infection_paths_no = len(infection_paths_known) + contact_no
    list_infection_paths_no.append(int(infection_paths_no))
print(list_infection_paths_no[:10])
print(list_infection_paths_no[-10:])

NameError: name 'list_infection_paths_no' is not defined

In [34]:
df_merged["no_paths"] = list_infection_paths_no
df_merged["new_confirmed"] = df_merged["new_confirmed"].apply(int)
df_merged["new_untracked"] = df_merged["new_untracked"].apply(int)
df_merged.head()

NameError: name 'list_infection_paths_no' is not defined

In [35]:
df_merged.tail()

,date,new_confirmed,new_untracked
220,2020-08-31,94.0,16.0
221,2020-09-01,101.0,18.0
222,2020-09-02,81.0,17.0
223,2020-09-03,62.0,11.0
224,2020-09-04,45.0,6.0


In [36]:
df_merged.to_csv("dataset_predict/df_predict.csv", index=False)

### check holiday

In [37]:
import json

list_calendar=[]

calendar_file = "./korean-calendar/korean-calendar.json"
# read file
with open(calendar_file, 'r') as myfile:
    data=myfile.readlines()

data = [x.strip() for x in data] 

for i in data:
    calendar_data = i[43:]
    calendar_item = "{"+calendar_data
    calendar_item_json = json.loads(calendar_item)
    list_calendar.append(calendar_item_json)
list_calendar[:10]

[{'sc': '2000-01-01', 'lc': '1999-11-25', 'w': 7, 'h': True, 'ht': '신정'},
 {'sc': '2000-01-02', 'lc': '1999-11-26', 'w': 1, 'h': False, 'ht': ''},
 {'sc': '2000-01-03', 'lc': '1999-11-27', 'w': 2, 'h': False, 'ht': ''},
 {'sc': '2000-01-04', 'lc': '1999-11-28', 'w': 3, 'h': False, 'ht': ''},
 {'sc': '2000-01-05', 'lc': '1999-11-29', 'w': 4, 'h': False, 'ht': ''},
 {'sc': '2000-01-06', 'lc': '1999-11-30', 'w': 5, 'h': False, 'ht': ''},
 {'sc': '2000-01-07', 'lc': '1999-12-01', 'w': 6, 'h': False, 'ht': ''},
 {'sc': '2000-01-08', 'lc': '1999-12-02', 'w': 7, 'h': False, 'ht': ''},
 {'sc': '2000-01-09', 'lc': '1999-12-03', 'w': 1, 'h': False, 'ht': ''},
 {'sc': '2000-01-10', 'lc': '1999-12-04', 'w': 2, 'h': False, 'ht': ''}]

In [38]:
df_calendar = pd.DataFrame(list_calendar)
df_calendar = df_calendar[["sc", "w", "h"]]
df_calendar.head()

,sc,w,h
0,2000-01-01,7,True
1,2000-01-02,1,False
2,2000-01-03,2,False
3,2000-01-04,3,False
4,2000-01-05,4,False


In [39]:
def label_weekend(row):
    # sunday
    if row['w'] == 1:
        return True
    # friday
    if row['w'] == 6:
        return True
    # saturday
    if row['w'] == 7:
        return True
    else:
        return False

In [40]:
df_calendar["is_weekend"] = df_calendar.apply(lambda row: label_weekend(row), axis=1)
df_calendar.head(10)

,sc,w,h,is_weekend
0,2000-01-01,7,True,True
1,2000-01-02,1,False,True
2,2000-01-03,2,False,False
3,2000-01-04,3,False,False
4,2000-01-05,4,False,False
5,2000-01-06,5,False,False
6,2000-01-07,6,False,True
7,2000-01-08,7,False,True
8,2000-01-09,1,False,True
9,2000-01-10,2,False,False


In [41]:
# rename dataframe name
df_calendar.rename(columns = {'sc':'date', 'h':'is_holiday'}, inplace = True)

In [42]:
df_calendar.to_csv("dataset_predict/df_calendar.csv", index=False)

In [43]:
df_calendar_subset = df_calendar[["date", "is_holiday", "is_weekend"]]

In [47]:
df_train = pd.merge(df_merged, df_calendar_subset, how="left", on="date")
df_train.tail()

,date,new_confirmed,new_untracked,is_holiday,is_weekend
220,2020-08-31,94.0,16.0,False,False
221,2020-09-01,101.0,18.0,False,False
222,2020-09-02,81.0,17.0,False,False
223,2020-09-03,62.0,11.0,False,False
224,2020-09-04,45.0,6.0,False,True


In [45]:
df_train[-25:-15]

,date,new_confirmed,new_untracked,is_holiday,is_weekend
200,2020-08-11,15.0,4.0,False,False
201,2020-08-12,26.0,5.0,False,False
202,2020-08-13,33.0,1.0,False,False
203,2020-08-14,73.0,4.0,False,True
204,2020-08-15,146.0,6.0,True,True
205,2020-08-16,90.0,4.0,False,True
206,2020-08-17,132.0,7.0,False,False
207,2020-08-18,151.0,18.0,False,False
208,2020-08-19,135.0,19.0,False,False
209,2020-08-20,126.0,28.0,False,False


In [46]:
df_train.to_csv("./dataset_predict/df_train.csv", index=False)

In [52]:
cols = list(df_train)[1:5]
dataset_train = df_train[cols].astype(str)
for i in cols:
    for j in range(0, len(dataset_train)):
        dataset_train[i][j] = dataset_train[i][j].replace(',', '')
        dataset_train[i][j] = dataset_train[i][j].replace('True', '1')
        dataset_train[i][j] = dataset_train[i][j].replace('False', '0')

dataset_train = dataset_train.astype(float)

dataset_train

,new_confirmed,new_untracked,is_holiday,is_weekend
0,1.0,0.0,1.0,1.0
1,0.0,0.0,1.0,1.0
2,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
220,94.0,16.0,0.0,0.0
221,101.0,18.0,0.0,0.0
222,81.0,17.0,0.0,0.0
223,62.0,11.0,0.0,0.0


In [57]:
dataset_train[-120:].corr()

,new_confirmed,new_untracked,is_holiday,is_weekend
new_confirmed,1.000000,0.871830,0.179294,-0.026664
new_untracked,0.871830,1.000000,-0.005647,-0.046515
is_holiday,0.179294,-0.005647,1.000000,0.148877
is_weekend,-0.026664,-0.046515,0.148877,1.000000


### Add Government Social Distancing Policy